# Show the data

In [1]:
import os 
os.chdir('iron_handmaidens')
import pandas as pd
from data.src.emg import EMGData
import plotly.graph_objects as go
import plotly.express as px

In [2]:
left = EMGData.read_csv('/Users/aldenmoreton/Documents/iron_handmaidens/test_data/participant1/A/Shimmer_4680.csv')
right = EMGData.read_csv('/Users/aldenmoreton/Documents/iron_handmaidens/test_data/participant1/A/Shimmer_5470.csv')
center = left + right

In [3]:
left.df[left[['CH']]] = left.normalize(left[['CH']])
left.df

,Timestamp_4680,Status_4680,CH1_4680,CH2_4680,Event_4680
0,1.635975e+12,128.0,0.072873,0.352086,0.0
1,1.635975e+12,128.0,0.073713,0.351705,-1.0
2,1.635975e+12,128.0,0.074364,0.351259,-1.0
3,1.635975e+12,128.0,0.074484,0.351721,-1.0
4,1.635975e+12,128.0,0.074787,0.353034,-1.0
...,...,...,...,...,...
1540876,1.635976e+12,128.0,0.120784,0.242968,-1.0
1540877,1.635976e+12,128.0,0.120819,0.242798,-1.0
1540878,1.635976e+12,128.0,0.120883,0.242781,-1.0
1540879,1.635976e+12,128.0,0.120807,0.242879,-1.0


In [4]:
left.preprocess()
left.figure().show()

In [5]:
left.quartiles()

,CH1_4680,CH2_4680
0.9,0.086624,0.440500
0.5,0.054300,0.297490
0.1,0.036617,0.205144


In [6]:
right.preprocess()
right.figure().show()

In [7]:
# center.preprocess()
center.maxDataPoints = 10000
center.figure().show()

In [8]:
center.quartiles()

,CH1_4680,CH2_4680,CH1_5470,CH2_5470
0.9,2.128309,2.714057,0.708234,4.752214
0.5,0.796343,0.593441,-0.395587,-0.512304
0.1,0.067915,-0.776029,-1.133812,-1.908459


In [9]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
df = px.data.gapminder().query("year == 2007")

fig = go.Figure()
# adding names for reference
fig.add_scatter(mode="markers", x=df["gdpPercap"], y=df["lifeExp"], 
                visible = False, name='linear')
fig.add_scatter(mode="markers", x=df["gdpPercap"], y=np.sqrt(df["lifeExp"]), 
                visible = False, name='sqrt')

# buttons for updatemenu
buttons = [dict(method='restyle',
                label='linear',
                visible=True,
                args=[{'label': 'linear',
                       'visible':[True, False],}]),
           dict(method='restyle',
                label='sqrt',
                visible=True,
                args=[{'label': 'linear',
                       'visible':[False, True],}])]
           
# specify updatemenu        
um = [{'buttons':buttons, 'direction': 'down'}]
fig.update_layout(updatemenus=um)

# Conditionally Updating Traces 
fig.for_each_trace(
    lambda trace: trace.update(visible=True) if trace.name == "linear" else (),
)
fig.add_vline(10000, name='Jeff')
fig.show()

In [10]:
left.df['Toggle'] = left.df[left['Event']].diff()
toggles = left.df.loc[abs(left.df['Toggle']) == 3]
toggles = toggles.reset_index()
toggles

,index,Timestamp_4680,Status_4680,CH1_4680,CH2_4680,Event_4680,Toggle
0,39268,1.635975e+12,128.0,0.116920,0.436131,2.0,3.0
1,45657,1.635975e+12,128.0,0.073698,0.463757,-1.0,-3.0
2,86663,1.635975e+12,128.0,0.103422,0.408375,2.0,3.0
3,94785,1.635975e+12,128.0,0.061858,0.417065,-1.0,-3.0
4,469831,1.635975e+12,128.0,0.050199,0.404598,2.0,3.0
5,1415977,1.635976e+12,128.0,0.048583,0.201196,-1.0,-3.0
6,1449111,1.635976e+12,128.0,0.036553,0.220943,2.0,3.0
7,1456384,1.635976e+12,128.0,0.057142,0.219168,-1.0,-3.0
8,1512873,1.635976e+12,128.0,0.104730,0.170547,2.0,3.0
9,1520821,1.635976e+12,128.0,0.094536,0.254649,-1.0,-3.0


In [12]:
testEMGData = left.preprocess()
fig = go.Figure()
plots = []
for line in testEMGData.find_columns('CH'):
	newFig = go.Scatter(x=testEMGData.df.iloc[::100][testEMGData['Elapse']], y=testEMGData.df[line].iloc[::100], name=line, visible='legendonly')
	plots.append(newFig)
[fig.add_trace(plot) for plot in plots]
fig.for_each_trace(lambda trace: trace.update(visible=True) if trace.name == 'CH1_4680' else ())

for i in range(0, len(toggles)-1, 2):
	fig.add_vrect(toggles.iloc[i][testEMGData['Elapse']], toggles.iloc[i+1][testEMGData['Elapse']], fillcolor='green', opacity=0.25)

fig.show()

KeyError: 'Elapse (s)'

In [14]:
def find_events(self):
	new = pd.DataFrame()

	self.df['Toggle'] = self.df[self['Event']].diff()
	new = self.df.loc[abs(self.df['Toggle']) == 3]
	new = new.reset_index()
	
	return new

In [17]:
find_events(right)

,index,Timestamp_5470,Status_5470,CH1_5470,CH2_5470,Event_5470,Toggle
0,39320,1.635975e+12,128.0,-1.108509,4.548831,2.0,3.0
1,45639,1.635975e+12,128.0,-0.162634,3.763427,-1.0,-3.0
2,86802,1.635975e+12,128.0,-0.594523,5.317407,2.0,3.0
3,94860,1.635975e+12,128.0,0.432369,4.349896,-1.0,-3.0
4,469667,1.635975e+12,128.0,0.835890,1.517559,2.0,3.0
5,1415871,1.635976e+12,128.0,-0.570242,-2.458506,-1.0,-3.0
6,1448983,1.635976e+12,128.0,-0.747901,-1.834534,2.0,3.0
7,1456273,1.635976e+12,128.0,-0.927003,0.658110,-1.0,-3.0
8,1512754,1.635976e+12,128.0,-1.551937,-2.502861,2.0,3.0
9,1520820,1.635976e+12,128.0,-1.208518,-1.961949,-1.0,-3.0


In [19]:
right.dtypes

AttributeError: 'EMGData' object has no attribute 'dtypes'